In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv


In [2]:
lyric_database = pd.read_csv("/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv")
lyric_database.head(5)

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [3]:
db = lyric_database.sample(frac=0.02, random_state=1) #this database has 3 million entries
db = (
    db
    .loc[db["language"] == "en"]
    .loc[db["tag"] != "misc"]
    .loc[db["tag"] != ""]
    .loc[db["tag"] != None]
    .loc[db["lyrics"] != None]
    .loc[db["lyrics"] != ""]
    .loc[db["lyrics"] != "[Instrumental]"]
)
columns_of_interest = ['id', 'title','artist', 'lyrics', 'tag']
db = db[columns_of_interest]
db = db.sort_values(by="id")
db.head(10)

,id,title,artist,lyrics,tag
9,11,Lord You Know,Cam'ron,"[Chorus: Jaheim]\nNow Lord you know, just how ...",rap
135,141,Evening News,Chamillionaire,"[Hook]\nGas prices raises, the money keeps bur...",rap
219,216,Get It In Ohio,Cam'ron,"[Intro]\nWhat up, Midwest?\nThey forgot about ...",rap
240,243,Fuck tha Police,N.W.A,[Produced by Dr. Dre & DJ Yella]\n\n[Intro: Th...,rap
280,263,Kick in the Door,The Notorious B.I.G.,"[Segue from ""Hypnotize""]\nBiggie, Biggie, Bigg...",rap
267,309,I Used to Love H.E.R.,Common,"[Produced by No I.D.]\n\n[Chorus]\nA-yes, yes ...",rap
340,334,Im Back,T.I.,"[Hook]\nI never let you down, I’ma shine on si...",rap
370,365,Downtown Swinga 98,M.O.P.,[Intro]\nYeah\nCome on\nInternational\nDowntow...,rap
382,376,Deep Cover,Dr. Dre,[Intro: Snoop Dogg & Cop]\nHit this motherfuck...,rap
388,383,La Menage,Black Sheep,"[Intro: Dres]\nListen, hon, I mean, ah, as sla...",rap


In [4]:
print(db.tag.unique())
print(db.tag.value_counts(normalize=True))

['rap' 'rock' 'rb' 'pop' 'country']
tag
pop        0.427210
rap        0.300285
rock       0.196880
rb         0.048093
country    0.027532
Name: proportion, dtype: float64


In [5]:
import re
import nltk

def fix_lyrics(lyrics):
    parts = re.split(r"([\n\[\]\(\)])", lyrics)
    output = " ".join(filter(None, parts))
    output = re.sub(r"([?.,!:;])",'',output)
    output = re.sub(r"in'(?= \w|\.|,|$)","ing",output.lower())
    return output


db["lyrics"] = db["lyrics"].apply(fix_lyrics)
db = db.sort_values(by="id")

In [6]:
from nltk.tokenize import word_tokenize
db['tokens_raw'] = db['lyrics'].apply(lambda x: word_tokenize(x.lower()))
db.head(10)

,id,title,artist,lyrics,tag,tokens_raw
9,11,Lord You Know,Cam'ron,[ chorus jaheim ] \n now lord you know just ho...,rap,"[[, chorus, jaheim, ], now, lord, you, know, j..."
135,141,Evening News,Chamillionaire,[ hook ] \n gas prices raises the money keeps ...,rap,"[[, hook, ], gas, prices, raises, the, money, ..."
219,216,Get It In Ohio,Cam'ron,[ intro ] \n what up midwest \n they forgot ab...,rap,"[[, intro, ], what, up, midwest, they, forgot,..."
240,243,Fuck tha Police,N.W.A,[ produced by dr dre & dj yella ] \n \n [ intr...,rap,"[[, produced, by, dr, dre, &, dj, yella, ], [,..."
280,263,Kick in the Door,The Notorious B.I.G.,"[ segue from ""hypnotize"" ] \n biggie biggie bi...",rap,"[[, segue, from, ``, hypnotize, '', ], biggie,..."
267,309,I Used to Love H.E.R.,Common,[ produced by no id ] \n \n [ chorus ] \n a-ye...,rap,"[[, produced, by, no, id, ], [, chorus, ], a-y..."
340,334,Im Back,T.I.,[ hook ] \n i never let you down i’ma shine on...,rap,"[[, hook, ], i, never, let, you, down, i, ’, m..."
370,365,Downtown Swinga 98,M.O.P.,[ intro ] \n yeah \n come on \n international ...,rap,"[[, intro, ], yeah, come, on, international, d..."
382,376,Deep Cover,Dr. Dre,[ intro snoop dogg & cop ] \n hit this motherf...,rap,"[[, intro, snoop, dogg, &, cop, ], hit, this, ..."
388,383,La Menage,Black Sheep,[ intro dres ] \n listen hon i mean ah as slam...,rap,"[[, intro, dres, ], listen, hon, i, mean, ah, ..."


In [7]:
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
db['tokens_raw'] = db['tokens_raw'].apply(lambda x: [w for w in x if w not in stopwords])

db.head(10)

,id,title,artist,lyrics,tag,tokens_raw
9,11,Lord You Know,Cam'ron,[ chorus jaheim ] \n now lord you know just ho...,rap,"[[, chorus, jaheim, ], lord, know, hard, try, ..."
135,141,Evening News,Chamillionaire,[ hook ] \n gas prices raises the money keeps ...,rap,"[[, hook, ], gas, prices, raises, money, keeps..."
219,216,Get It In Ohio,Cam'ron,[ intro ] \n what up midwest \n they forgot ab...,rap,"[[, intro, ], midwest, forgot, fourth, coast, ..."
240,243,Fuck tha Police,N.W.A,[ produced by dr dre & dj yella ] \n \n [ intr...,rap,"[[, produced, dr, dre, &, dj, yella, ], [, int..."
280,263,Kick in the Door,The Notorious B.I.G.,"[ segue from ""hypnotize"" ] \n biggie biggie bi...",rap,"[[, segue, ``, hypnotize, '', ], biggie, biggi..."
267,309,I Used to Love H.E.R.,Common,[ produced by no id ] \n \n [ chorus ] \n a-ye...,rap,"[[, produced, id, ], [, chorus, ], a-yes, yes,..."
340,334,Im Back,T.I.,[ hook ] \n i never let you down i’ma shine on...,rap,"[[, hook, ], never, let, ’, shine, sight, keep..."
370,365,Downtown Swinga 98,M.O.P.,[ intro ] \n yeah \n come on \n international ...,rap,"[[, intro, ], yeah, come, international, downt..."
382,376,Deep Cover,Dr. Dre,[ intro snoop dogg & cop ] \n hit this motherf...,rap,"[[, intro, snoop, dogg, &, cop, ], hit, mother..."
388,383,La Menage,Black Sheep,[ intro dres ] \n listen hon i mean ah as slam...,rap,"[[, intro, dres, ], listen, hon, mean, ah, sla..."


In [8]:
import random
random.seed(42)

db_full = db.copy()

indices = list(range(db_full.shape[0]))

random.shuffle(indices)

cutoff = int(db_full.shape[0]*0.75)
train_indices = indices[:cutoff]
test_indices = indices[cutoff:]

# df will be the in-sample training dataset
db = db_full.iloc[train_indices,:].copy()
print(f"db.shape: {db.shape}")

# df_test will be the out-of-sample validation dataset
db_test = db_full.iloc[test_indices,:].copy()
print(f"db_test.shape: {db_test.shape}")
db_test.head()

db.shape: (48406, 6)
db_test.shape: (16136, 6)


,id,title,artist,lyrics,tag,tokens_raw
2516942,3792631,Dancer,Tim the Lion Tamer,i've never seen nobody \n dance like you \n in...,pop,"['ve, never, seen, nobody, dance, like, times,..."
146529,158361,Billy Bibbit,Taktik,here voluntarily where a nurse walks me down \...,rap,"[voluntarily, nurse, walks, white, halls, corr..."
803591,1118915,James Joint,Rihanna,[ verse 1 ] \n i'd rather be smoking weed when...,pop,"[[, verse, 1, ], 'd, rather, smoking, weed, wh..."
2193906,3274566,Never Knew Until Now,Sybyr,[ verse 1 ] \n you know me now but what about ...,rap,"[[, verse, 1, ], know, back, bullshit, happene..."
3975786,6011314,Stay Alive Pt. 2,BERGDORFBADMAN,intro ( bluef7ame ) \n & i can’t leave you b...,rap,"[intro, (, bluef7ame, ), &, ’, leave, baby, dr..."


In [9]:
db_pop = db[db['tag'] == 'pop']
db_rap = db[db['tag'] == 'rap']
db_rock = db[db['tag'] == 'rock']
db_rb = db[db['tag'] == 'rb']
db_country = db[db['tag'] == 'country']

def create_vocab_list(tokens_column):
    vocab = dict()
    for tokens in tokens_column:
        for token in tokens:
            if token not in vocab:
                vocab[token] = 1
            else:
                vocab[token] += 1
    return vocab

vocab_all = dict(sorted(create_vocab_list(db['tokens_raw']).items(), key=lambda item: item[1], reverse=True))
vocab_pop = dict(sorted(create_vocab_list(db_pop['tokens_raw']).items(), key=lambda item: item[1], reverse=True))
vocab_rap = dict(sorted(create_vocab_list(db_rap['tokens_raw']).items(), key=lambda item: item[1], reverse=True))
vocab_rock = dict(sorted(create_vocab_list(db_rock['tokens_raw']).items(), key=lambda item: item[1], reverse=True))
vocab_rb = dict(sorted(create_vocab_list(db_rb['tokens_raw']).items(), key=lambda item: item[1], reverse=True))
vocab_country = dict(sorted(create_vocab_list(db_country['tokens_raw']).items(), key=lambda item: item[1], reverse=True))
print(f"number of unique tokens overall: {len(vocab_all)}\npop tokens: {len(vocab_pop)}\nrap tokens: {len(vocab_rap)}\
      \nrock tokens: {len(vocab_rock)}\nrb tokens: {len(vocab_rb)}\ncountry tokens: {len(vocab_country)}")

top_n_to_remove = 100 #500 #500 #1000 # for our ad hoc model we can think of this as a type of hyperparameter
for i, item in enumerate(vocab_all.items()):
    if i == top_n_to_remove:
        break
    #print(f" removing token: {item[0]:15} (w/ freq = {item[1]:5}) from vocabs")
    if item[0] in vocab_pop:
        del vocab_pop[item[0]]
    if item[0] in vocab_rap:
        del vocab_rap[item[0]]
    if item[0] in vocab_rock:
        del vocab_rock[item[0]]
    if item[0] in vocab_rb:
        del vocab_rb[item[0]]
    if item[0] in vocab_country:
        del vocab_country[item[0]]

list(vocab_all.items())[:20]

number of unique tokens overall: 139789
pop tokens: 59394
rap tokens: 101754      
rock tokens: 36539
rb tokens: 17616
country tokens: 11081


[("n't", 172389),
 ('[', 144363),
 (']', 144338),
 ("'s", 143345),
 ('’', 135666),
 (')', 125832),
 ('(', 125814),
 ("'m", 122761),
 ('like', 101361),
 ('know', 83055),
 ('got', 82659),
 ('get', 60413),
 ('love', 59394),
 ('yeah', 58745),
 ('na', 55117),
 ('verse', 49905),
 ('go', 46144),
 ("'re", 45721),
 ('oh', 42553),
 ('see', 42364)]